# VCP Pattern Scanner - Stock Market Technical Analysis

This notebook scans for **Vertical Consolidation Pattern (VCP)** opportunities across multiple global markets:
- US **US Markets**: S&P 500 + Nasdaq (508 stocks)
- HK **Hong Kong**: HSI + HSTECH Combined (103 stocks)
- JP **Japan**: MSCI Japan Index (180 stocks)
- Tech **Tech Sector**: Global tech stocks

**VCP Pattern**: Progressive wave contractions forming a tightening range, indicating potential breakout setup.

---

## 1. Setup: Install Dependencies

In [ ]:
import subprocess
import sys

packages = ['yfinance', 'pandas', 'numpy', 'requests', 'rich', 'openpyxl']
print('Installing dependencies...')
subprocess.check_call([sys.executable, '-m', 'pip', 'install', '-q'] + packages)
print('Dependencies installed!')

## 2. Download: VCP Scanner & Watchlists from GitHub

In [ ]:
import os

os.makedirs('watchlists', exist_ok=True)
os.makedirs('Reports', exist_ok=True)

print('Downloading VCP_scanner.py...')
os.system('wget -q https://raw.githubusercontent.com/clementwai-sketch/Spring/main/VCP_Scanner/VCP_scanner.py')

base_url = 'https://raw.githubusercontent.com/clementwai-sketch/Spring/main/VCP_Scanner/watchlists/'
watchlists = ['hk_stocks.json', 'us_stocks.json', 'japan_stocks.json', 'tech_sector.json']

print('Downloading watchlist files...')
for watchlist in watchlists:
    os.system(f'wget -q {base_url}{watchlist} -O watchlists/{watchlist}')
    print(f'  Done: {watchlist}')

print('\nAll files downloaded!')
os.system('ls -lh VCP_scanner.py watchlists/')

## 3. Select: Choose Your Watchlist

Use the dropdown menu below to select your target market.

In [ ]:
import ipywidgets as widgets
from IPython.display import display

# Dropdown menu for watchlist selection
watchlist_dropdown = widgets.Dropdown(
    options=[
        ('1 - Tech Sector (Global Tech Stocks)', 1),
        ('2 - US Markets (S&P 500 + Nasdaq)', 2),
        ('3 - Japan MSCI (180 Stocks)', 3),
        ('4 - Hong Kong (HSI + HSTECH Combined)', 4),
    ],
    value=1,
    description='Watchlist:',
    style={'description_width': '80px'},
    layout=widgets.Layout(width='400px')
)

display(watchlist_dropdown)
print(f'\nSelected: Option {watchlist_dropdown.value}')

## 4. Scan: Run VCP Pattern Detection

In [ ]:
import subprocess
import sys

# Read the current dropdown selection
WATCHLIST_CHOICE = watchlist_dropdown.value
names = {1: 'Tech Sector', 2: 'US Markets', 3: 'Japan MSCI', 4: 'Hong Kong'}

print(f'Running VCP scanner: {names[WATCHLIST_CHOICE]} (Option {WATCHLIST_CHOICE})')
print('=' * 60)

result = subprocess.run(
    [sys.executable, 'VCP_scanner.py'],
    input=f'{WATCHLIST_CHOICE}\n',
    text=True,
    capture_output=False
)

print('=' * 60)
print('Scan complete!')

## 5. Results: View & Export Findings

In [ ]:
import os

print('Generated Reports:')
print('=' * 60)

if os.path.exists('Reports'):
    files = sorted(os.listdir('Reports'), reverse=True)
    if files:
        for file in files[:5]:
            filepath = os.path.join('Reports', file)
            size = os.path.getsize(filepath) / 1024
            print(f'  {file} ({size:.1f} KB)')
    else:
        print('  No reports generated yet. Try running the scanner above.')
else:
    print('  Reports directory not found. Run the scanner first.')

print('\nTip: Open .csv files with pandas for analysis, .xlsx for spreadsheet view')

In [ ]:
import pandas as pd
import os

# Load and display latest CSV results
if os.path.exists('Reports'):
    csv_files = [f for f in os.listdir('Reports') if f.endswith('.csv')]
else:
    csv_files = []

if csv_files:
    latest_csv = sorted(csv_files, reverse=True)[0]
    filepath = os.path.join('Reports', latest_csv)
    print(f'Latest Results: {latest_csv}\n')

    df = pd.read_csv(filepath)
    print(f'Found {len(df)} VCP patterns')
    print('\n' + '=' * 60)

    if 'Quality Score' in df.columns:
        prime = df[df['Quality Score'] > 70].sort_values('Quality Score', ascending=False)
        if len(prime) > 0:
            print(f'PRIME VCP PATTERNS (Quality > 70): {len(prime)}')
            display_cols = [col for col in df.columns if col not in ['Analysis', 'Details']]
            print(prime[display_cols].head(10).to_string())
        else:
            print('No Prime VCP patterns found this scan.')

    print('\nFull dataset available in df variable for further analysis')
else:
    print('No CSV files found. Run the scanner cell above first.')

## 6. Advanced: Configuration Reference

### VCP Detection Parameters

| Parameter | Value | Purpose |
|-----------|-------|----------|
| **MIN_CONTRACTIONS** | 3 | Minimum wave contractions required |
| **MAX_CONTRACTIONS** | 6 | Maximum waves before pattern breaks |
| **CONTRACTION_RATIO** | 0.70 | Each wave must be <=70% of previous |
| **MAX_LAST_CONTRACTION** | 12.0% | Final squeeze tightness threshold |
| **MIN_PRICE_ABOVE_MA** | 3.0% | Uptrend confirmation (price > 50-day MA) |
| **TREND_MA_PERIOD** | 50 | Moving average for trend filter |
| **VOLUME_MA_PERIOD** | 50 | Moving average for volume analysis |

### Market-Specific Settings

| Market | Zigzag % | Benchmark |
|--------|----------|----------|
| **US** (SPX, NQ) | 5.0% | ^GSPC, ^IXIC |
| **Japan** (.T suffix) | 4.0% | ^N225 |
| **Hong Kong** (.HK suffix) | 6.0% | ^HSI |
| **China** (.SS, .SZ) | 7.0% | FXI |

### Quality Score Breakdown

- **Contractions** (25%): Number and progression of waves
- **Trendiness** (20%): Price positioning vs 50-day MA
- **Volume** (20%): Average volume and recent trends
- **Tightness** (15%): How compressed the final wave is
- **Relative Strength** (20%): Performance vs market benchmark

Score above **70** = Prime Setup | Score 50-70 = Watch List

---

## Documentation

**GitHub Repository**: [clementwai-sketch/Spring](https://github.com/clementwai-sketch/Spring)

**Files**:
- VCP_scanner.py - Main detection engine
- watchlists/hk_stocks.json - Hong Kong stocks
- watchlists/us_stocks.json - US market (SPX + NQ)
- watchlists/japan_stocks.json - Japan MSCI
- watchlists/tech_sector.json - Global tech stocks

**Output**:
- Reports/VCP_Report_*.csv - Detection results with quality scores
- Reports/VCP_Report_*.xlsx - Formatted spreadsheet with charts

**Note**: This is a technical analysis tool for educational purposes. Always perform your own due diligence before trading.